<a href="https://colab.research.google.com/github/ruina272/pieces-os/blob/main/sakura_14b_qwen2_5_v1_0_q6k_gguf_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Run API backend (vLLM)
# Use ngrok for API mapping (leave ngrokToken blank to use localtunnel)
from huggingface_hub import hf_hub_download

ngrokToken = "2jWQyAuEx9IIAypszGWVyjwLpta_7rZaeVi77PNnEZZ2oVqJN"  # @param {type:"string"}
if ngrokToken:
  from pyngrok import conf, ngrok
  conf.get_default().auth_token = ngrokToken
  conf.get_default().monitor_thread = False
  ssh_tunnels = ngrok.get_tunnels(conf.get_default())
  if len(ssh_tunnels) == 0:
      ssh_tunnel = ngrok.connect(8001)
      public_url = ssh_tunnel.public_url
      print('address：'+public_url)
  else:
      public_url = ssh_tunnels[0].public_url
      print('address：'+public_url)
else:
  import subprocess
  import threading
  def start_localtunnel(port):
      p = subprocess.Popen(["npx", "localtunnel", "--port", f"{port}"], stdout=subprocess.PIPE)
      for line in p.stdout:
          print(line.decode(), end='')
  threading.Thread(target=start_localtunnel, daemon=True, args=(8001,)).start()
  public_url = "http://localhost:8001"

%cd $ROOT_PATH

# Download the model for the first time
!HF_ENDPOINT=https://huggingface.co huggingface-cli download SakuraLLM/Sakura-14B-Qwen2.5-v1.0-GGUF --local-dir models --include sakura-14b-qwen2.5-v1.0-q6k.gguf

!RAY_memory_monitor_refresh_ms="0" HF_ENDPOINT=https://huggingface.co OMP_NUM_THREADS=36 VLLM_ATTENTION_BACKEND=XFORMERS vllm serve ./models/sakura-14b-qwen2.5-v1.0-q6k.gguf --tokenizer Qwen/Qwen2.5-14B-Instruct --dtype float16 --api-key token-abc123 --kv-cache-dtype auto --max-model-len 4096 --tensor-parallel-size 1 --gpu-memory-utilization 0.99 --disable-custom-all-reduce --enforce-eager --use-v2-block-manager --disable-log-requests --host 0.0.0.0 --port 8001 --served-model-name "Qwen2.5-14B-Instruct" &

print(f"API is accessible at {public_url}")

address：https://a23a-34-127-37-203.ngrok-free.app
[Errno 2] No such file or directory: '$ROOT_PATH'
/content
Fetching 1 files:   0% 0/1 [00:00<?, ?it/s]Downloading 'sakura-14b-qwen2.5-v1.0-q6k.gguf' to 'models/.cache/huggingface/download/sakura-14b-qwen2.5-v1.0-q6k.gguf.2c1fc22a43c15cc42cf1443822115b979c2f9143671f5369df0158d9fc184b77.incomplete'

sakura-14b-qwen2.5-v1.0-q6k.gguf:   0% 0.00/12.1G [00:00<?, ?B/s]
sakura-14b-qwen2.5-v1.0-q6k.gguf:   0% 10.5M/12.1G [00:00<05:04, 39.8MB/s]
sakura-14b-qwen2.5-v1.0-q6k.gguf:   0% 21.0M/12.1G [00:00<04:51, 41.5MB/s]
sakura-14b-qwen2.5-v1.0-q6k.gguf:   0% 31.5M/12.1G [00:00<04:46, 42.3MB/s]
sakura-14b-qwen2.5-v1.0-q6k.gguf:   0% 41.9M/12.1G [00:00<04:43, 42.6MB/s]
sakura-14b-qwen2.5-v1.0-q6k.gguf:   0% 52.4M/12.1G [00:01<04:43, 42.6MB/s]
sakura-14b-qwen2.5-v1.0-q6k.gguf:   1% 62.9M/12.1G [00:01<04:41, 42.8MB/s]
sakura-14b-qwen2.5-v1.0-q6k.gguf:   1% 73.4M/12.1G [00:01<04:42, 42.7MB/s]
sakura-14b-qwen2.5-v1.0-q6k.gguf:   1% 83.9M/12.1G [00:01<04

In [1]:
#@title # Keep this widget playing to prevent Colab from disconnecting you { display-mode: "form" }
#@markdown Press play on the audio player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [2]:
# Mount Google Drive
Mount_GDrive = True # @param {type:"boolean"}
if Mount_GDrive:
  from google.colab import drive
  drive.mount('/content/gdrive')
  ROOT_PATH = "/content/gdrive/MyDrive"
else:
  ROOT_PATH = "/content"
!nvidia-smi

Mounted at /content/gdrive
Thu Nov 28 12:04:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+----------------------------------------

In [1]:
# Install dependencies
!pip install --upgrade pip
!pip install transformers tokenizers
!pip install vllm huggingface-hub flask pyngrok triton torch